In [47]:
from rethinkdb import RethinkDB
# import os
from random import randint
import mimesis
from mimesis.enums import Gender

r = RethinkDB()

## Working with demo DB

### Creating demo DB

In [48]:
conn = r.connect()
exist = False
for i in r.db_list().run(conn):
    if i == 'demo':
        exist = True
if not exist:
    r.db_create('demo').run(conn)
conn = r.connect(db="demo")

### Creating sample table and inserting values

In [49]:
exist = False
for i in r.db_list().run(conn):
    if i == 'demo':
        exist = True
if not exist:
    r.table_create('sample', primary_key='id').run(conn)
    
r.table("sample").insert({
    "title": "Lorem ipsum",
    "content": "Dolor sit amet"
}).run(conn)    
print (r.table('sample').run(conn))
# You can see that primary key 'id' was auto-generated

rethinkdb.net.DefaultCursor (done streaming): 
[{'content': 'Dolor sit amet',
  'id': '6a3aeb23-2e98-4c1f-80bb-f99ea7e9b6e8',
  'title': 'Lorem ipsum'},
 {'content': 'Dolor sit amet',
  'id': 'ae1c2f04-c453-4b33-8689-e5b462fd9096',
  'title': 'Lorem ipsum'},
 {'color': 'blue',
  'id': '8ab73f6a-e381-48a9-ae53-0f6972b0dd2a',
  'shape': 'Dolor sit amet'},
 {'color': 'blue',
  'id': 'f3fabbf3-2508-46a7-94e4-5369d23650f4',
  'shape': 'Dolor sit amet'}]


### Geospatial commands examples 

### We will try to insert new value in table 'sample' with completely different fields

In [50]:
r.table("sample").insert({
    "color": "blue",
    "shape": "circle"
}).run(conn)
print(r.table('sample').run(conn))

rethinkdb.net.DefaultCursor (done streaming): 
[{'content': 'Dolor sit amet',
  'id': '6a3aeb23-2e98-4c1f-80bb-f99ea7e9b6e8',
  'title': 'Lorem ipsum'},
 {'content': 'Dolor sit amet',
  'id': 'ae1c2f04-c453-4b33-8689-e5b462fd9096',
  'title': 'Lorem ipsum'},
 {'color': 'blue',
  'id': '7c5c140a-58c2-49fa-adbe-6722f1c9ffa5',
  'shape': 'circle'},
 {'color': 'blue',
  'id': '8ab73f6a-e381-48a9-ae53-0f6972b0dd2a',
  'shape': 'Dolor sit amet'},
 {'color': 'blue',
  'id': 'f3fabbf3-2508-46a7-94e4-5369d23650f4',
  'shape': 'Dolor sit amet'}]


### Geospatial commands example

In [51]:
point1 = r.point(154.562564, 37.779388)
point2 = r.point(-117.220406, 32.719464)
print(r.distance(point1, point2, unit='km').run(conn))

7733.8894794419075


## Examples of work with main DB

In [52]:
conn = r.connect(db="HMS")
print('Connection installed.')

Connection installed.


### List of tables in 'HMS' DB

In [53]:
print(r.db('HMS').table_list().run(conn))

['Administrators', 'Ambulances', 'Departments', 'Doctors', 'Hospital', 'IllnessForms', 'IllnessHistories', 'Nurses', 'Paramedics', 'Patients', 'Stuff']


### Manipulations with 'Doctors' table

In [54]:
print(r.table("Doctors").run(conn))

rethinkdb.net.DefaultCursor (done streaming): 
[{'SSN_ID': 8285985750577324,
  'date_of_birth': '1970-09-22',
  'doctor_id': 4,
  'home_address': '910 Balceta Point',
  'name': 'Edmundo Rivas',
  'qualification ': 'Pediatric surgery',
  'salary': 12000,
  'sex': 'male',
  'telephone': 89174309036},
 {'SSN_ID': 3097837436296817,
  'date_of_birth': '1977-03-22',
  'doctor_id': 3,
  'home_address': '1006 Ivy Hill',
  'name': 'Carson Ewing',
  'qualification ': 'Thoracic surgery',
  'salary': 12000,
  'sex': 'male',
  'telephone': 89608530557},
 {'SSN_ID': 8601208044784384,
  'date_of_birth': '1970-03-24',
  'doctor_id': 0,
  'home_address': '830 Santa Ana Parade',
  'name': 'Augustus Franklin',
  'qualification ': 'Thoracic surgery',
  'salary': 12000,
  'sex': 'male',
  'telephone': 89735653163},
 {'SSN_ID': 1597241025243178,
  'date_of_birth': '1955-05-07',
  'doctor_id': 1,
  'home_address': '772 Clairview Private',
  'name': 'Lamont Benson',
  'qualification ': 'Pediatric surgery',
  

In [55]:
print(r.table('Doctors').filter({"qualification ":  "Pediatric surgery"}).run(conn))

rethinkdb.net.DefaultCursor (done streaming): 
[{'SSN_ID': 8285985750577324,
  'date_of_birth': '1970-09-22',
  'doctor_id': 4,
  'home_address': '910 Balceta Point',
  'name': 'Edmundo Rivas',
  'qualification ': 'Pediatric surgery',
  'salary': 12000,
  'sex': 'male',
  'telephone': 89174309036},
 {'SSN_ID': 1597241025243178,
  'date_of_birth': '1955-05-07',
  'doctor_id': 1,
  'home_address': '772 Clairview Private',
  'name': 'Lamont Benson',
  'qualification ': 'Pediatric surgery',
  'salary': 12000,
  'sex': 'male',
  'telephone': 89347240402}]


### Amount of records in 'Patients' table

In [56]:
r.table('Patients').count().run(conn)

100

### All women patients with their names starting with 'A-C'

In [57]:
r.table('Patients').filter(lambda sex: sex['sex'] == 'female').filter(lambda name:
    name['name'].match("^[A-C]")
).run(conn)

<rethinkdb.net.DefaultCursor object at 0x7fd1adab0d68 (done streaming): 
[{'SSN_ID': 6105738676170308,
  'coordinates': [807.56, 919.27],
  'date_of_birth': '1972-10-06',
  'home_address': '1120 Mansell Plaza',
  'illness_history_head_id': 30,
  'name': 'Christinia Mcclain',
  'patient_id': 30,
  'sex': 'female',
  'telephone': 89337887993},
 {'SSN_ID': 4137291608511207,
  'coordinates': [1209.8, 1078.38],
  'date_of_birth': '1961-06-25',
  'home_address': '57 Rex Highway',
  'illness_history_head_id': 95,
  'name': 'Aleida Garcia',
  'patient_id': 95,
  'sex': 'female',
  'telephone': 89220008859},
 {'SSN_ID': 4790548940838920,
  'coordinates': [409.34, 786.33],
  'date_of_birth': '1983-07-01',
  'home_address': '1199 Quane Mews',
  'illness_history_head_id': 53,
  'name': 'Bernetta Talley',
  'patient_id': 53,
  'sex': 'female',
  'telephone': 89975265160}]>